### HEAD

In [105]:
import math
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import functools
import operator

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from functools import reduce
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from typing import List, Callable, Tuple
import warnings
warnings.filterwarnings("ignore")

In [3]:
path = r"C:\Users\repin\Desktop\GitHub\TestLab\development\test-data.csv"
df = pd.read_csv(path)

In [4]:
# добавим данные по месяцу и дате в удобном формате
df['month'] = df['Date'].apply(lambda x: x[:7])
df['dt'] = df['Date'].apply(lambda x: x[:10])

In [5]:
# введем переменные с датами
# пусть предэкспериментальный период будет равен 50 дням (чуть больше теста)
end_of_30 = datetime.datetime.strptime('2022-04-07', "%Y-%m-%d")
start_of_30 = end_of_30 + datetime.timedelta(days=-50)

start_dt = start_of_30.strftime("%Y-%m-%d")
end_dt = end_of_30.strftime("%Y-%m-%d")
end_of_test_dt = '2022-05-19'
start_dt, end_dt, end_of_test_dt

('2022-02-16', '2022-04-07', '2022-05-19')

In [6]:
# данные предэкспериментального периода
pre_exp = df[
    (df['dt'] >= start_dt)&(df['dt'] < end_dt)
    &(df['OrderSource'] == 'web')&(df['Category'] == 'пицца')
    ]
pre_exp.shape

(13140, 14)

In [7]:
pre_exp = pre_exp[["dt", "CityName", "rto"]]
pre_exp = pre_exp[~pre_exp["CityName"].isin(["Москва", "Санкт-Петербург"])]

In [8]:
pre_exp_pivot = pre_exp.pivot_table(values="rto", columns="dt", index="CityName")

In [9]:
cols = list(pre_exp_pivot.columns)

wrong_cities = []
for i in cols:
    one_row = pre_exp_pivot.loc[:, i]
    if one_row.isnull().values.any():
        all_cities = list(one_row[one_row.isnull().values].index)
        for i in all_cities:
            if i not in wrong_cities:
                wrong_cities.append(i)

In [10]:
# оставляем города, включая мск и спб, кроме тех, где есть пропуски в данных
test_wo_denied_ = [i for i in set(pre_exp["CityName"]) if i not in wrong_cities]

In [11]:
len(test_wo_denied_)

228

In [58]:
knn_pre_exp = pre_exp[["dt", "CityName", "rto"]]
knn_pre_exp_clean = knn_pre_exp[knn_pre_exp["CityName"].isin(test_wo_denied_)]

In [59]:
knn_pre_exp_clean.head()

,dt,CityName,rto
139295,2022-02-16,Абакан,19586.21
139307,2022-02-16,Абинск,7909.93
139319,2022-02-16,Азов,2019.43
139329,2022-02-16,Аксай Россия,2657.27
139340,2022-02-16,Александров,7047.18


### FUNCs

In [104]:
def get_scaled_data(data: pd.DataFrame, metric_name: str) -> pd.DataFrame:
    """
    Метод для масштабирования данных. Используется StandardScaler

    Args:
        data (pd.DataFrame): датафрейм с датами, юнитами и метрикой
        metric_name (str): наменование столбца с метрикой

    Raises:
        KeyError: В случае, если наименование столбца с метрикой указано
        неверно, бросается исключение

    Returns:
        pd.DataFrame: исходный датафрейм + масштабированная метрика
    """
    try:
        scaled_metric = StandardScaler().fit_transform(data[[metric_name]])
    except KeyError:
        raise KeyError(f"Frame data does not contain the field with name {metric_name}")
    data[f"scaled_{metric_name}"] = scaled_metric
    return data

In [61]:
knn_pre_exp_clean_scld = get_scaled_data(knn_pre_exp_clean, "rto")

In [62]:
knn_pre_exp_clean_scld.head()

,dt,CityName,rto,scaled_rto
139295,2022-02-16,Абакан,19586.21,-0.316794
139307,2022-02-16,Абинск,7909.93,-0.549947
139319,2022-02-16,Азов,2019.43,-0.667569
139329,2022-02-16,Аксай Россия,2657.27,-0.654832
139340,2022-02-16,Александров,7047.18,-0.567174


In [106]:
def get_vectors(data: pd.DataFrame, metric_name: str, id_field_name: str) -> Tuple[dict, dict]:
    """
    Преобразует метрику из датафрейма в векторный вид

    Args:
        data (pd.DataFrame): датафрейм c масштабированной метрикой
        metric_name (str): наименование столбца с масштабированной метрикой
        id_field_name (str): наименование столбца с юнитом

    Returns:
        Tuple[dict, dict]:
            dict0 - словарь с наименованием юнита в ключе и вектором в значении
            dict1 - словарь с индексом юнита в ключе и наименованием юнита в значении
    """
    data_vec = data.groupby(id_field_name).agg({metric_name: list}).reset_index()
    data_vec[f"{metric_name}_array"] = [np.array(i) for i in data_vec[f"{metric_name}"]]
    keys = data_vec[id_field_name].tolist()
    vals = data_vec[f"{metric_name}_array"].tolist()
    return dict(zip(keys, vals)), dict(zip([i for i in range(0, len(keys))], keys))

In [88]:
knn_vectors, ids_dict = get_vectors(knn_pre_exp_clean_scld, "scaled_rto", "CityName")

In [107]:
def get_k_neighbours(
    id: str, vectors: dict, number_of_neighbours: int, algorithm='auto'
) -> dict:
    """
    Возвращает k ближайших соседей для одного заданного юнита

    Args:
        id (str): идентификатор юнита
        vectors (dict): словарь с наименованием юнита в ключе и вектором метрики в значении 
        number_of_neighbours (int): количество ближайших соседей для поиска
        algorithm (str, optional): алгорит подбора соседей. Defaults to 'auto'.

    Returns:
        dict: словарь с индексами юнитов в ключах и расстоянием в значении
    """
    def get_knn(vectors):
        vector_arrays = [list(i) for i in vectors.values()]
        return NearestNeighbors(number_of_neighbours, algorithm=algorithm).fit(vector_arrays)        

    def get_vector(vectors, id):
        return vectors[id].reshape(1, -1)

    def flatten_neighbour_list(distance, ids):
        dist_list, nb_list = distance.tolist(), ids.tolist()
        return dist_list[0], nb_list[0]

    knn = get_knn(vectors)
    vector = get_vector(vectors, id)
    dist, nb_indexes = knn.kneighbors(vector, number_of_neighbours, return_distance=True)
    return_dist, return_nb_indexes = flatten_neighbour_list(dist, nb_indexes)
    return dict(zip(return_nb_indexes, return_dist))

In [81]:
get_k_neighbours("Орск", knn_vectors, 3)

{147: 0.0, 76: 0.767695996137587, 83: 0.8031629508608411}

In [82]:
test_towns = ['Пенза', 'Уфа', 'Курск', 'Нижний Тагил', 'Новокуйбышевск', 'Орск']

In [108]:
def get_all_neighbours(
    knn_vectors: dict, ids_dict: dict, test_towns: List[str], number_of_neighbours: int
) -> dict:
    """
    Возвращает словарь с ближайшими соседями для всех, поданных на вход юнитов

    Args:
        knn_vectors (dict): словарь с наименованием юнита в ключе и вектором в значении
        ids_dict (dict): словарь с индексом юнита в ключе и наименованием юнита в значении
        test_towns (List[str]): список юнитов из тестовой группы
        number_of_neighbours (int): количество ближайших соседей

    Returns:
        dict: словарь с наменованием юнита в ключе и списком соседей в значении
    """
    result_ids = {
        i:[ids_dict[j] for j in get_k_neighbours(i, knn_vectors, number_of_neighbours)]
        for i in test_towns
    }
    return result_ids

In [97]:
all_neighbours = get_all_neighbours(knn_vectors, ids_dict, test_towns, 2)
all_neighbours

{'Пенза': ['Пенза', 'Рязань'],
 'Уфа': ['Уфа', 'Оренбург'],
 'Курск': ['Курск', 'Ухта'],
 'Нижний Тагил': ['Нижний Тагил', 'Брянск'],
 'Новокуйбышевск': ['Новокуйбышевск', 'Выборг'],
 'Орск': ['Орск', 'Ковров']}

In [111]:
def get_test_control_val_groups(neighbours_dict: dict) -> dict:
    """
    Формирует словарь со списками тестовых и контрольных юнитов в значениях словаря

    Args:
        neighbours_dict (dict): словарь с наменованием юнита в ключе и списком соседей в значении

    Returns:
        dict: итоговый словарь {test_units: [str], control_units: [str]}
    """
    test_units = list(neighbours_dict.keys())
    control_units = [[j for j in i if j not in test_towns][0] for i in neighbours_dict.values()]
    return dict(
        test_units=test_units,
        control_units=control_units,
    )

In [110]:
all_groups = get_test_control_val_groups(all_neighbours)
all_groups

{'test_units': ['Пенза',
  'Уфа',
  'Курск',
  'Нижний Тагил',
  'Новокуйбышевск',
  'Орск'],
 'control_units': ['Рязань', 'Оренбург', 'Ухта', 'Брянск', 'Выборг', 'Ковров']}